## Ablation study on KS dataset for PFNN latent dimensions

In [ ]:
cd ..

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from model.utilities import *
from model.vae_base import *
import sys
sys.path.append('./model')
from sklearn.decomposition import PCA

import numpy.random as random
from scipy.stats import gaussian_kde


font = {'size'   : 12, 'family': 'Times New Roman'}
matplotlib.rc('font', **font)

In [2]:
torch.manual_seed(0)
np.random.seed(0)

# Main
n_train = 900
n_test = 100

batch_size = 100

sub = 4 # spatial subsample
S = 512
s = S//sub

T_in = 500 # skip first 100 seconds of each trajectory to let trajectory reach attractor
T = 1100 # seconds to extract from each trajectory in data
T_out = T_in + T
step = 1 # Seconds to learn solution operator

# Load data
predloader = MatReader('./data/KS.mat') # load the generated data
data_raw = predloader.read_field('u')
data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]

# randomly sample half episodes from the train data episodes
episode_samples = int(0.5*n_train)
data_sampled_train = data_tensor[torch.randperm(data_tensor[:n_train].size(0))[:episode_samples],:,:]
# data_sampled_test = data_tensor[torch.randperm(data_tensor[-n_test:].size(0)),:,:]
data_test = data_tensor[-n_test:,:,:]

train_sample = data_sampled_train[:,T_in:T_out,:].reshape(-1, s)
test_a = data_test[:,T_in-1:T_out-1,:].reshape(-1, s)
test_u = data_test[:,T_in:T_out,:].reshape(-1, s)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=False)

/tmp/ipykernel_3678717/605233945.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]


In [ ]:
device = torch.device('cpu')

PFNN_latent_1_path = 'fill_PFNN_latent_128_model_path'
model_latent_1 = torch.load(PFNN_latent_1_path, map_location=device)

PFNN_latent_2_path = 'fill_PFNN_latent_256_model_path'
model_latent_2 = torch.load(PFNN_latent_2_path, map_location=device)

PFNN_latent_4_path = 'fill_PFNN_latent_512_model_path'
model_latent_4 = torch.load(PFNN_latent_4_path, map_location=device)

PFNN_latent_8_path = 'fill_PFNN_latent_1024_model_path'
model_latent_8 = torch.load(PFNN_latent_8_path, map_location=device)


In [4]:
def long_prediction(model, test_a, id, steps_per_sec, out_dim, in_dim, koopman = False, ae=False, T=T):
      model.eval()
      # id = id*T
      test_a_0 = test_a[id]

      pred = torch.zeros(T, out_dim)
      out = test_a_0.reshape(1,in_dim).to(device)
      with torch.no_grad():
            for i in range(T):
                  if koopman:
                        out = model(out, mode='forward')[0][0]
                  elif ae:
                        out = model(out.reshape(1,in_dim))[0]
                  else:
                        out = model(out.reshape(1,in_dim,1))
                  pred[i] = out.view(out_dim,)
      
      return pred

In [5]:
# Step 1: Instantiate PCA object
# To keep all components but see how many are needed for 95% variance, you can set n_components to 0.95
pca = PCA(n_components=5)

# Step 2: Fit PCA on the data
pca.fit(np.concatenate((train_sample, test_u), axis=0))

# Step 3: Transform the data to get the principal components
test_u_pca = pca.transform(test_u)

# The transformed data `test_u_pca` now has fewer dimensions, with 95% of the original variance retained
print("Original shape:", test_u.shape)
print("Reduced shape:", test_u_pca.shape)

# If you want to see the number of components selected
print("Number of components selected:", pca.n_components_)

Original shape: torch.Size([110000, 128])
Reduced shape: (110000, 5)
Number of components selected: 5


In [6]:
reduced_model1_long_pred = []
reduced_model2_long_pred = []
reduced_model3_long_pred = []
reduced_model4_long_pred = []

with torch.no_grad():
      for init_id in range(n_test):
            latent_1_long_pred = long_prediction(model_latent_1, test_a, init_id*T, 1, s, s, ae=True)
            latent_2_long_pred = long_prediction(model_latent_2, test_a, init_id*T, 2, s, s, ae=True)
            latent_4_long_pred = long_prediction(model_latent_4, test_a, init_id*T, 4, s, s, ae=True)
            latent_8_long_pred = long_prediction(model_latent_8, test_a, init_id*T, 8, s, s, ae=True)


            reduced_model1_long_pred.append(pca.transform(latent_1_long_pred))
            reduced_model2_long_pred.append(pca.transform(latent_2_long_pred))
            reduced_model3_long_pred.append(pca.transform(latent_4_long_pred))
            reduced_model4_long_pred.append(pca.transform(latent_8_long_pred))


reduced_model1_long_pred = np.concatenate(reduced_model1_long_pred, axis=0)
reduced_model2_long_pred = np.concatenate(reduced_model2_long_pred, axis=0)
reduced_model3_long_pred = np.concatenate(reduced_model3_long_pred, axis=0)
reduced_model4_long_pred = np.concatenate(reduced_model4_long_pred, axis=0)

### Calculate KL divergence of principle compoennets' distribution 

In [8]:
kde_truth_pca = calculate_kde(pca.transform(np.concatenate([train_sample, test_u])))
kde_latent_1_pca = calculate_kde(reduced_model1_long_pred)
kde_latent_2_pca = calculate_kde(reduced_model2_long_pred)
kde_latent_4_pca = calculate_kde(reduced_model3_long_pred)
kde_latent_8_pca = calculate_kde(reduced_model4_long_pred)
x_range = np.linspace(np.concatenate([train_sample, test_u]).min()-5, np.concatenate([train_sample, test_u]).max()+5, 40000)

In [9]:
kl_dim_tru_latent_1 = []
kl_dim_tru_latent_2 = []
kl_dim_tru_latent_4 = []
kl_dim_tru_latent_8 = []


for i in range(5):
      print(f"Dimension {i+1}")

      kl_dim_tru_latent_1_i = kl_divergence(kde_truth_pca[i], kde_latent_1_pca[i], x_range)
      kl_dim_tru_latent_1.append(kl_dim_tru_latent_1_i)
      print(f"KL divergence (PC distribution) True vs latent_1*128: {kl_dim_tru_latent_1_i}")

      kl_dim_tru_latent_2_i = kl_divergence(kde_truth_pca[i], kde_latent_2_pca[i], x_range)
      kl_dim_tru_latent_2.append(kl_dim_tru_latent_2_i)
      print(f"KL divergence (PC distribution) True vs latent_2*128: {kl_dim_tru_latent_2_i}")

      kl_dim_tru_latent_4_i = kl_divergence(kde_truth_pca[i], kde_latent_4_pca[i], x_range)
      kl_dim_tru_latent_4.append(kl_dim_tru_latent_4_i)
      print(f"KL divergence (PC distribution) True vs latent_4*128: {kl_dim_tru_latent_4_i}")

      kl_dim_tru_latent_8_i = kl_divergence(kde_truth_pca[i], kde_latent_8_pca[i], x_range)
      kl_dim_tru_latent_8.append(kl_dim_tru_latent_8_i)
      print(f"KL divergence (PC distribution) True vs latent_8*128: {kl_dim_tru_latent_8_i}")


Dimension 1
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.00011974 0.00012005 0.00012035 ... 0.00125472 0.00125447 0.00125422] q_x shape: 40000
KL divergence (PC distribution) True vs latent_1: 83.4974924409619
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.00026265 0.00026304 0.00026342 ... 0.00033662 0.00033624 0.00033586] q_x shape: 40000
KL divergence (PC distribution) True vs latent_2: 28.03252220172805
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.00066794 0.00066803 0.00066812 ... 0.00021954 0.00021935 0.00021916] q_x shape: 40000
KL divergence (PC distribution) True vs latent_4: 4.281563488371166
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.0132704  0.01327096 0.01327153 ... 0.06419277 0.06418683 0.06418089] q_x shape: 40000
KL divergence (PC distribution) T